In [1]:
!pip install python-dotenv
!pip install openai
!pip install numPy


In [2]:
import os
from openai import AzureOpenAI

#read .env variables
from dotenv import load_dotenv
load_dotenv()

True

In [12]:

# // Cell 3 - Fix the api_version variable
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
embeddings_deployment = os.getenv("AZURE_EMBEDDINGS_DEPLOYMENT_NAME", "text-embedding-ada-002")

print("AZURE_OPENAI_ENDPOINT:", azure_endpoint)
print("Chat Deployment:", deployment_name)
print("Embeddings Deployment:", embeddings_deployment)

AZURE_OPENAI_ENDPOINT: https://5350.openai.azure.com
Chat Deployment: gpt-4.1-mini
Embeddings Deployment: text-embedding-ada-002


In [13]:
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
)

In [14]:

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_completion_tokens=13107,
    temperature=1.0,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    model= deployment_name,
)

print(response.choices[0].message.content)

That sounds wonderful! Paris is full of incredible sights and experiences. Here are some must-see attractions and activities:

1. **Eiffel Tower** – The iconic symbol of Paris. Consider going up for amazing views of the city.
2. **Louvre Museum** – Home to thousands of artworks, including the Mona Lisa and the Venus de Milo.
3. **Notre-Dame Cathedral** – A stunning example of Gothic architecture (currently under restoration but the exterior can be admired).
4. **Sacré-Cœur Basilica** – Located on Montmartre hill, offering panoramic views over Paris.
5. **Champs-Élysées and Arc de Triomphe** – Famous avenue for shopping and the impressive arch commemorating French soldiers.
6. **Seine River Cruise** – A relaxing way to see many landmarks from the water.
7. **Musée d'Orsay** – Beautiful museum featuring Impressionist art in a former railway station.
8. **Luxembourg Gardens** – A lovely park perfect for a stroll or picnic.
9. **Le Marais District** – Trendy neighborhood with historic buil

In [15]:
response = client.chat.completions.create(
    model=deployment_name, 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)
print(response.choices[0].message.content)

Yes, many other Azure AI services also support customer-managed keys (CMKs). Services such as Azure Cognitive Services—including Azure Text Analytics, Azure Computer Vision, and Azure Speech—allow you to use CMKs to encrypt your data at rest. This enables you to have greater control over encryption keys used to protect your data, enhancing security and compliance.

However, support for customer-managed keys can vary by service and region, so it’s a good idea to check the specific documentation for each Azure AI service you plan to use to confirm CMK support and any configuration details.


In [ ]:
# Calculate vector embeddings for classic book titles and find similarities
import numpy as np
from numpy.linalg import norm

# Classic book titles to compare
book_titles = [
    "Pride and Prejudice",
    "Sense and Sensibility",
    "War and Peace",
    "Crime and Punishment",
    "The Brothers Karamazov",
    "Moby Dick",
    "The Great Gatsby",
    "1984",
    "Animal Farm",
    "To Kill a Mockingbird"
]

# Get embeddings for each book title
embeddings = []
for title in book_titles:
    response = client.embeddings.create(
        model=embeddings_deployment,  # You may need to adjust this to your deployment name
        input=title
    )
    embeddings.append(response.data[0].embedding)

# Convert to numpy array for easier computation
embeddings_array = np.array(embeddings)

# Calculate cosine similarity between all pairs
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

# Find and display the most similar book pairs
print("Most Similar Book Titles (based on embeddings):\n")
similarities = []

for i in range(len(book_titles)):
    for j in range(i + 1, len(book_titles)):
        similarity = cosine_similarity(embeddings_array[i], embeddings_array[j])
        similarities.append((book_titles[i], book_titles[j], similarity))

# Sort by similarity (highest first)
similarities.sort(key=lambda x: x[2], reverse=True)

# Display top 10 most similar pairs
for i, (book1, book2, sim) in enumerate(similarities[:10], 1):
    print(f"{i}. '{book1}' ↔ '{book2}'")
    print(f"   Cosine Similarity: {sim:.4f}\n")

Most Similar Book Titles (based on embeddings):

1. 'The Great Gatsby' ↔ 'To Kill a Mockingbird'
   Cosine Similarity: 0.8598

2. 'Pride and Prejudice' ↔ 'Sense and Sensibility'
   Cosine Similarity: 0.8564

3. 'War and Peace' ↔ 'The Brothers Karamazov'
   Cosine Similarity: 0.8500

4. 'Moby Dick' ↔ 'To Kill a Mockingbird'
   Cosine Similarity: 0.8498

5. 'Pride and Prejudice' ↔ 'To Kill a Mockingbird'
   Cosine Similarity: 0.8459

6. 'Crime and Punishment' ↔ 'The Brothers Karamazov'
   Cosine Similarity: 0.8450

7. 'Pride and Prejudice' ↔ 'The Great Gatsby'
   Cosine Similarity: 0.8436

8. 'Pride and Prejudice' ↔ 'War and Peace'
   Cosine Similarity: 0.8404

9. 'Moby Dick' ↔ 'The Great Gatsby'
   Cosine Similarity: 0.8401

10. 'Crime and Punishment' ↔ 'To Kill a Mockingbird'
   Cosine Similarity: 0.8399

